# Download the inventory data for the following criterias
Select an earthquake event (by id).\
Get the station inventory within the study area which recorded this event.\
Use this station inventory data to download earthquake data (`.mseed`) and station data (`.txt` or `.xml` format).\
3-channel data for earthquakes are along x, y and z axis. Usually named as HHE, HHN and HHZ or HH1, HH2, HHZ.\
I want to select the following channels in order. If the 1st one is available, get that one only and stop. If not search for the 2nd one and so on.
1. HH*
2. BH*
3. HN*
4. EH* \
Lastly, 


In [1]:
import os
import time
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, read_inventory, Inventory, read, Stream

In [3]:
# Read earthquake data
eqdf = pd.read_csv("../data/above_slab_eq_df.csv", parse_dates=["time"])
eqdf4 = eqdf[eqdf.mag >= 4].reset_index(drop=True)

# Extract unique grid codes
unique_grid_codes = eqdf4['grid_code'].unique()

# Loop through each grid code
for i in range(len(unique_grid_codes)):
    temp_df = eqdf4[eqdf4.grid_code == unique_grid_codes[i]].reset_index(drop=True) # select events in the grid

    # Check if there are events in the grid
    if len(temp_df) > 0:
        print(temp_df[['id', 'mag', 'time']])
        # Select the event with maximum magnitude
        eq_idx = temp_df['mag'].idxmax()
        print(f"Selected event index: {eq_idx}")

        # get the details of the event
        eq = temp_df.iloc[eq_idx]
        event_id = eq['id']
        print(f"Selected event: {event_id}, mag: {eq['mag']} in grid {unique_grid_codes[i]}")

        # if an event is already downloaded, skip
        if os.path.exists(f"../data/eq_data/{event_id}/inventory/station_inventory_{event_id}.xml"):
            print(f"Event {event_id} already downloaded. Skipping...")
            continue

        # Get event time
        event_time = UTCDateTime(pd.to_datetime(eq.time))
        starttime = event_time - 30
        endtime = event_time + 120

        # define the datacenters and channel list
        client_list = ['IRIS', 'NCEDC', 'SCEDC']
        channel_list = 'HH*,BH*,HN*,EH*' # select broadband and high sample rate channels

        # Create a folder for the event
        output_folder = f"../data/eq_data/{event_id}/inventory"
        os.makedirs(output_folder, exist_ok=True)

        merged_inventory = Inventory()

        # Loop through each client (IRIS, NCEDC, SCEDC data centers)
        for client_name in client_list:
            client = Client(client_name, debug=False, timeout=3600)
            try:
                inv = client.get_stations(
                    network="*",
                    station="*",
                    location="*",
                    channel=channel_list,
                    starttime=starttime,
                    endtime=endtime,
                    level="channel",
                    minlatitude=39.75,
                    maxlatitude=41.5,
                    minlongitude=-125,
                    maxlongitude=-123,
                )
                merged_inventory.networks.extend(inv.networks)
                merged_inventory = merged_inventory.remove(network="SY")

                # Write the inventory to files
                # inv.write(f"{output_folder}/{event_id}_{client_name}.xml", format="STATIONXML")
                # inv.write(f"{output_folder}/{event_id}_{client_name}.txt", format="STATIONTXT")

            except Exception as e:
                print(f"Error fetching data from {client_name}: {e}")    

        break # test with only one grid
        
        # write the whole inventory to a file
        merged_inventory.write(f"{output_folder}/station_inventory_{event_id}.xml", format="STATIONXML")
        
    else:
        print(f"No events in grid code {unique_grid_codes[i]}")
        continue


           id   mag                              time
0  nc72086051  4.94  2013-10-11 23:05:37.330000+00:00
Selected event index: 0
Selected event: nc72086051, mag: 4.94 in grid 3
Event nc72086051 already downloaded. Skipping...
           id   mag                              time
0  nc73139366  4.19  2019-02-03 23:43:21.810000+00:00
1  nc73139111  4.30  2019-02-02 10:52:21.990000+00:00
2  nc73201181  5.58  2019-06-23 03:53:02.890000+00:00
3  nc73133256  4.08  2019-01-15 20:20:20.120000+00:00
4  nc72820761  4.03  2017-06-24 21:22:03.060000+00:00
Selected event index: 2
Selected event: nc73201181, mag: 5.58 in grid 6


# Read the inventory files already downloaded 
# And download `.mseed` and station_data (`.xml & .txt`)

Here I will read the inventory file which contain details about all the stations that recorded a particular earthquake event. \
From that inventory file I will get all the necessary informations I need to download the seismic data (a numpy timeseries in `.mseed` format). \
I will also download the metadata for that record in `.xml & .txt` formats.\

This process will use `multiprocessing.Pool.imap_unordered` module for paraller processing of the download.\
For the code see `./code/my_funcs/get_waveforms_parallel_v3.py` where I defined the download fuction combined with parallel processing. \
This significantly improves the runtime.


In [11]:
# reload the module to get the latest changes
import sys
sys.path.append('./my_funcs')
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import glob
import os 
# import all the `get_waveforms` function
from my_funcs.get_waveforms_parallel_v3 import *

# define the client list i.e. the data centers to download data from
client_list = ['NCEDC', 'IRIS'] #, 'SCEDC']

# get a list of all the event id folders
event_paths = glob.glob("../data/eq_data/*")
event_ids = [os.path.basename(path) for path in event_paths]

# Read earthquake data
eqdf = pd.read_csv("../data/above_slab_eq_df.csv", parse_dates=["time"])

# define the priority channels
priority_channels = ['HH*', 'BH*', 'HN*', 'EH*']

event_ids = ['nc73201181'] # test with one event #################################### change it ####################
# loop through each event id and download the data
for event_id in event_ids:

    # define the output folder
    output_folder = f"../data/eq_data/{event_id}/"

    # check if the event data is already downloaded
    if os.path.exists(f"../data/eq_data/{event_id}/{event_id}.mseed"):
        print(f"Event {event_id} already downloaded. Skipping...")
        continue

    print(f"Getting data for event {event_id}")

    # Read the inventory
    inventory = read_inventory(f"../data/eq_data/{event_id}/inventory/station_inventory_{event_id}.xml")

    #get the event time, start time and end time
    eq = eqdf[eqdf.id == event_id] # get the event details
    event_time = UTCDateTime(pd.to_datetime(eq.time.values[0])) # get the event time in UTC format
    starttime = event_time - 30 # start time is 30 seconds before the event time
    endtime = event_time + 120 # end time is 120 seconds after the event time

    # Call the function with the desired parameters
    # this will downaload and write the data to a file, to change path, edit the function
    get_waveforms_parallel(client_list, inventory, starttime, endtime, output_folder, priority_channels)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Getting data for event nc73201181
Fetching data from NCEDC...
Fetching data from IRIS...


100%|██████████| 108/108 [00:25<00:00,  4.24it/s]
/Users/mdarifulislam/miniconda3/envs/obspy/lib/python3.10/site-packages/obspy/io/mseed/core.py:823: UserWarning: File will be written with more than one different record lengths.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'record lengths')
